# TAREFA 6 - Algoritmo Genético

<ul>
<li>Diego Santos Gonçalves - 20183012537</li>
<li>Mariana Bulgarelli Alves dos Santos - 20183000330</li>
</ul>

Para executar este notebook, é necessária a instalação das seguintes bibliotecas:
<ul>
<li>numpy </li>
<li>pandas</li>
<li>random </li>
<li>matplotlib - para criação dos gráficos </li>
<li>mpl_toolkits - para criação dos gráficos 3d</li>
<li>tqdm - para barra progresso </li>
</ul>

In [ ]:
from tqdm import tqdm
import math
import random
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d

## Função Objetivo
Nós implementamos a função objetivo dada para o problema: função alpine 02. Neste caso nossa função terá apenas duas entradas (x1, x2).

In [ ]:
# Funcao Objetivo
def f_alpine02(x):
    resultado = 1
    for i, x_i in x.iteritems():
        resultado = resultado * math.sin(x_i) * math.sqrt(x_i)
    return resultado

In [ ]:
def eval(populacao): # fitness
    arr = []
    limiar = 10
    for i, row in populacao.iterrows():
        resultado = f_alpine02(row) + limiar
        arr.append(resultado)
    return arr    

In [ ]:
def selecionar(populacao, n1,f): #seleciona o maior fitness
    populacao['aux'] = f
    populacao = populacao.sort_values('aux', ascending=False).drop('aux', axis = 1)
    return populacao.reset_index(drop=True).head(n1)

In [ ]:
def clone(P1, f, n_clones): # proporcional aos anticorpos
    C = pd.DataFrame()
    for i, row in P1.iterrows():
        for j in range(n_clones):
            C.concat(row)
    return C.reset_index(drop=True)

In [ ]:
def mutacao(C, f, p, n_clones): 
    arr = []
    for i, row in C.iterrow():
        d = f[int(i/n_clones)]
        d_max = max(f)
        d_star = d / d_max
        alpha = math.exp(-p*d_star)
        line = []
        for j, xj in row.iteritems():
            rand = np.random.random()
            if rand < alpha:
                xj = np.random.uniform(0.0, 10.0)
            line.append(xj)
        arr.append(line)
    C1 = pd.DataFrame(data=arr, columns=['X1', 'X2'])
    return C1

In [ ]:
# Parametros do algoritmo imunologico
n1 = 50
n2 = 0
beta = 0.1
n_clones = round(beta*n1)
p = 10
max_iteracoes = 50

# Cria dataframe e preenche com valores randomicos
populacao = pd.DataFrame(np.random.uniform(0.0, 10.0, size=(n1, 2)), columns=['X1', 'X2'])

for t in tqdm(range(max_iteracoes)): 
    f = eval(populacao.copy()) # vetor
    P1 = selecionar(populacao.copy(), n1,f)
    C = clone(P1, f, n_clones)
    C1 = mutacao(C1, f, p, n_clones)
    f1 = eval(C1)
    P1 = selecionar(C1, n1, f1)
    P = substituir(P, n2)


   
    for i in range(0, int(tamanho_pop/2)):
        # chama selecao
        P1 = selecao_roleta(Fx, populacao_ordenada)
        # para que não haja repetição de individuos
        while(True):
            P2 = selecao_roleta(Fx, populacao_ordenada)
            
            if( P2['X1'] == P1['X1'] and P2['X2'] == P1['X2']):
                break

        # transforma P1 e P2 em vetor
        P1 = P1.to_numpy()
        P2 = P2.to_numpy()

        # faz crossover
        C1, C2 = crossover(P1, P2, tx_crossover)

        # Realizar mutacao de C1 e C2
        C1 = mutacao_uniforme(C1, tx_mutacao)
        C2 = mutacao_uniforme(C2, tx_mutacao)
        
        nova_populacao=nova_populacao.append(pd.DataFrame(data=[C1, C2], columns = ['X1', 'X2'] ))
 
    populacao = nova_populacao.reset_index(drop=True)
    plot_geracao_3d(populacao,geracao)




print("==========================================")
print("Best individuo: ")
print(best)
print("Funcao alpine02: ",f_alpine02(best))
print("Geracao: ",melhor_geracao)

In [ ]:
def plot_geracao_3d(df, geracao):
    populacao = df.copy()
    populacao['fx'] = populacao.apply(f_alpine02, axis=1)
    if geracao%10 == 0:
        print("Geracao: ", geracao)
        fig = plt.figure(figsize= (16,9))
        ax = plt.axes(projection = '3d')
        
        ax.grid(b = True, color = 'grey', linestyle = '-.', linewidth = 0.3, alpha = 0.2)
        
        my_cmap = plt.get_cmap('hsv')
        
        sctt = ax.scatter3D(populacao['X1'],populacao['X2'], populacao["fx"], alpha = 0.8, c=populacao["fx"], cmap= my_cmap, marker = '.')
        
        plt.title(f"Funcao alpine2 - Geracao {geracao}")
        ax.set_xlabel('x1')
        ax.set_ylabel('x2')
        ax.set_zlabel('f(x1,x2)')
        fig.colorbar(sctt, ax=ax, shrink= 0.5, aspect = 5)
        
        plt.show()
        plt.clf()
        plt.close()